In [9]:
# =========================
# 01_data_loading.ipynb
# =========================

# Imports & display settings
import os
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)
pd.set_option("display.float_format", "{:,.4f}".format)

# -------------------------
# Paths (relative to notebooks/)
# -------------------------
RAW_DIR = os.path.join("..", "data", "raw")
PROCESSED_DIR = os.path.join("..", "data", "processed")

FREQ_PATH = os.path.join(RAW_DIR, "freMTPL2freq.csv")
SEV_PATH  = os.path.join(RAW_DIR, "freMTPL2sev.csv")

print("RAW_DIR:", os.path.abspath(RAW_DIR))
print("PROCESSED_DIR:", os.path.abspath(PROCESSED_DIR))
print("FREQ_PATH exists?", os.path.exists(FREQ_PATH))
print("SEV_PATH exists?", os.path.exists(SEV_PATH))

# If these print False, your CSVs aren't in data/raw yet.
# We'll handle the "files too big" GitHub upload issue by downloading via script later if needed.

# -------------------------
# Load raw datasets
# -------------------------
# Note: low_memory=False avoids dtype guessing warnings in some CSVs
freq = pd.read_csv(FREQ_PATH, low_memory=False)
sev  = pd.read_csv(SEV_PATH, low_memory=False)

# -------------------------
# Sanity checks
# -------------------------
assert not freq.empty, "Frequency dataset is empty"
assert not sev.empty, "Severity dataset is empty"

print("\n✅ Loaded datasets")
print("Frequency shape:", freq.shape)
print("Severity shape:", sev.shape)

# -------------------------
# Quick preview
# -------------------------
display(freq.head())
display(sev.head())

# -------------------------
# Schema inspection
# -------------------------
print("\n--- Frequency info ---")
display(freq.info())

print("\n--- Severity info ---")
display(sev.info())

print("\nFrequency columns:\n", list(freq.columns))
print("\nSeverity columns:\n", list(sev.columns))

# -------------------------
# Expected column checks (MTPL standard)
# -------------------------
expected_freq_cols = {"IDpol", "Exposure", "ClaimNb"}
expected_sev_cols  = {"IDpol", "ClaimAmount"}

missing_freq = expected_freq_cols.difference(freq.columns)
missing_sev  = expected_sev_cols.difference(sev.columns)

assert len(missing_freq) == 0, f"Missing frequency columns: {missing_freq}"
assert len(missing_sev) == 0, f"Missing severity columns: {missing_sev}"

print("\n✅ Expected columns present")

# -------------------------
# Basic data quality checks
# -------------------------
print("\n--- Missing value rate (top 10) ---")
display(freq.isnull().mean().sort_values(ascending=False).head(10))
display(sev.isnull().mean().sort_values(ascending=False).head(10))

# Exposure should be > 0
bad_exposure = (freq["Exposure"] <= 0).sum()
print("\nBad Exposure (<=0) count:", bad_exposure)

# ClaimNb should be non-negative
neg_claims = (freq["ClaimNb"] < 0).sum()
print("Negative ClaimNb count:", neg_claims)

# ClaimNb should be integer-like (sometimes stored as float but whole-valued)
non_integer_claimnb = (~np.isclose(freq["ClaimNb"], np.round(freq["ClaimNb"]))).sum()
print("Non-integer ClaimNb count:", non_integer_claimnb)

# -------------------------
# Distribution snapshots
# -------------------------
print("\n--- ClaimNb distribution (normalized) ---")
display(freq["ClaimNb"].value_counts(normalize=True).sort_index().head(20))

print("\n--- ClaimAmount describe ---")
display(sev["ClaimAmount"].describe())

print("\n--- ClaimAmount quantiles ---")
display(sev["ClaimAmount"].quantile([0.5, 0.9, 0.99, 0.999]))

# Basic skew intuition
skew_val = sev["ClaimAmount"].skew()
print("\nClaimAmount skew:", skew_val)

# -------------------------
# Save base tables to processed/
# -------------------------
os.makedirs(PROCESSED_DIR, exist_ok=True)

FREQ_OUT = os.path.join(PROCESSED_DIR, "frequency_base.csv")
SEV_OUT  = os.path.join(PROCESSED_DIR, "severity_base.csv")

freq.to_csv(FREQ_OUT, index=False)
sev.to_csv(SEV_OUT, index=False)

print("\n✅ Saved processed base tables:")
print(" -", os.path.abspath(FREQ_OUT))
print(" -", os.path.abspath(SEV_OUT))

# -------------------------
# Summary (printed)
# -------------------------
print(
"""
Summary
- Loaded MTPL frequency and severity datasets
- Verified schema, exposure validity, and claim logic
- Observed strong zero-inflation in claim counts (many ClaimNb == 0)
- Confirmed heavy right-skew in claim severity (ClaimAmount)
- Saved clean base tables for EDA and modeling
"""
)

RAW_DIR: /Users/shamsrupak/Documents/insurance-risk-and-claims-modeling/data/raw
PROCESSED_DIR: /Users/shamsrupak/Documents/insurance-risk-and-claims-modeling/data/processed
FREQ_PATH exists? True
SEV_PATH exists? True

✅ Loaded datasets
Frequency shape: (678013, 12)
Severity shape: (26639, 2)


,IDpol,ClaimNb,Exposure,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Area,Density,Region
0,1.0000,1,0.1000,5,0,55,50,B12,Regular,D,1217,Rhone-Alpes
1,3.0000,1,0.7700,5,0,55,50,B12,Regular,D,1217,Rhone-Alpes
2,5.0000,1,0.7500,6,2,52,50,B12,Diesel,B,54,Picardie
3,10.0000,1,0.0900,7,0,46,50,B12,Diesel,B,76,Aquitaine
4,11.0000,1,0.8400,7,0,46,50,B12,Diesel,B,76,Aquitaine


,IDpol,ClaimAmount
0,1552,995.2000
1,1010996,"1,128.1200"
2,4024277,"1,851.1100"
3,4007252,"1,204.0000"
4,4046424,"1,204.0000"



--- Frequency info ---
<class 'pandas.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   IDpol       678013 non-null  float64
 1   ClaimNb     678013 non-null  int64  
 2   Exposure    678013 non-null  float64
 3   VehPower    678013 non-null  int64  
 4   VehAge      678013 non-null  int64  
 5   DrivAge     678013 non-null  int64  
 6   BonusMalus  678013 non-null  int64  
 7   VehBrand    678013 non-null  str    
 8   VehGas      678013 non-null  str    
 9   Area        678013 non-null  str    
 10  Density     678013 non-null  int64  
 11  Region      678013 non-null  str    
dtypes: float64(2), int64(6), str(4)
memory usage: 62.1 MB


None

...


--- ClaimAmount describe ---


count      26,639.0000
mean        2,278.5364
std        29,297.4813
min             1.0000
25%           686.8100
50%         1,172.0000
75%         1,228.0800
max     4,075,400.5600
Name: ClaimAmount, dtype: float64


--- ClaimAmount quantiles ---


0.5000     1,172.0000
0.9000     2,799.0720
0.9900    16,793.7044
0.9990   162,784.4424
Name: ClaimAmount, dtype: float64


ClaimAmount skew: 109.56449249616011

✅ Saved processed base tables:
 - /Users/shamsrupak/Documents/insurance-risk-and-claims-modeling/data/processed/frequency_base.csv
 - /Users/shamsrupak/Documents/insurance-risk-and-claims-modeling/data/processed/severity_base.csv

Summary
- Loaded MTPL frequency and severity datasets
- Verified schema, exposure validity, and claim logic
- Observed strong zero-inflation in claim counts (many ClaimNb == 0)
- Confirmed heavy right-skew in claim severity (ClaimAmount)
- Saved clean base tables for EDA and modeling



In [10]:
import os
print(os.listdir("../data/raw")[:10])

['freMTPL2freq.csv', 'freMTPL2sev.csv', '.gitkeep']
